In [6]:
import numpy as np
from scipy import stats
import pandas as pd

#tengo 20 archivos resultX.csv que quiero leer
dataset_real = []
dataset_sintetico = []
for i in range(20):
    #leo el archivo resultX.csv
    df = pd.read_csv(f"results{i}.csv")
    #me quiero quedar solo las que en la columna "Dataset" tiene Gaussian, Atom, Chainlink, Target, TwoDiamonds, Wine, BreastCancer, SAHeart, Iris, Stroke
    df_real = df[df["Dataset"].isin([ "Wine", "BreastCancer", "SAHeart", "Iris", "Stroke"])]
    df_sintetico = df[df["Dataset"].isin([ "Gaussian", "Atom", "Chainlink", "Target", "TwoDiamonds"])]
    #dropeamos los casos en que en columna Silhouette tenemos un valor 0
    df_real = df_real[df_real["Silhouette"] != 0]
    df_sintetico = df_sintetico[df_sintetico["Silhouette"] != 0]
    #agrego el dataframe a la lista
    dataset_real.append(df_real)
    dataset_sintetico.append(df_sintetico)
    
    
df_real = pd.concat(dataset_real)
df_sintetico = pd.concat(dataset_sintetico)

In [7]:
algorithms = df_real["Algorithm"].unique()
for algorithm in algorithms:
    #me quedo solo con el algoritmo
    df_real_algorithm = df_real[df_real["Algorithm"] == algorithm]
    df_sintetico_algorithm = df_sintetico[df_sintetico["Algorithm"] == algorithm]
    
    #calculamos el intervalo de confianza para el algoritmo de la columna Silhouette
    #real
    mean_real = df_real_algorithm["Silhouette"].mean()
    std_real = df_real_algorithm["Silhouette"].std()
    n_real = df_real_algorithm["Silhouette"].count()
    #sintetico
    mean_sintetico = df_sintetico_algorithm["Silhouette"].mean()
    std_sintetico = df_sintetico_algorithm["Silhouette"].std()
    n_sintetico = df_sintetico_algorithm["Silhouette"].count()
    #intervalo de confianza
    confidence = 0.95
    z = stats.norm.ppf((1 + confidence) / 2)
    #real
    se_real = std_real / np.sqrt(n_real)
    ci_real = (mean_real - z * se_real, mean_real + z * se_real)
    #sintetico
    se_sintetico = std_sintetico / np.sqrt(n_sintetico)
    ci_sintetico = (mean_sintetico - z * se_sintetico, mean_sintetico + z * se_sintetico)
    #imprimo el resultado
    print(f"Algorithm: {algorithm}")
    print(f"Real: {mean_real:.4f} ({std_real:.4f}) [{n_real}] {ci_real[0]:.4f} - {ci_real[1]:.4f}", min(df_real_algorithm["Silhouette"]), max(df_real_algorithm["Silhouette"]))
    print(f"Sintetico: {mean_sintetico:.4f} ({std_sintetico:.4f}) [{n_sintetico}] {ci_sintetico[0]:.4f} - {ci_sintetico[1]:.4f}", min(df_sintetico_algorithm["Silhouette"]), max(df_sintetico_algorithm["Silhouette"]))
    print()

Algorithm: KMeans
Real: 0.3722 (0.0889) [40] 0.3447 - 0.3998 0.2806817957373884 0.4724920211075079
Sintetico: 0.4439 (0.1347) [100] 0.4175 - 0.4703 0.2476330627755211 0.6314108025017449

Algorithm: DBSCAN
Real: 0.3565 (0.0000) [20] 0.3565 - 0.3565 0.3565164814270073 0.3565164814270073
Sintetico: 0.2764 (0.1157) [80] 0.2510 - 0.3017 0.1536358301003274 0.4608703145263482

Algorithm: Agglomerative
Real: 0.3621 (0.0857) [40] 0.3355 - 0.3886 0.2774439826952266 0.4466890410285909
Sintetico: 0.4560 (0.1299) [100] 0.4306 - 0.4815 0.2719429637104548 0.6314707303920268

Algorithm: CDSDG Clustering
Real: 0.3712 (0.1357) [40] 0.3292 - 0.4133 0.2142867164487772 0.504645610832545
Sintetico: 0.4635 (0.0511) [100] 0.4535 - 0.4735 0.4213864174626807 0.6154700887378831

Algorithm: CDSDG Voting
Real: 0.2661 (0.0300) [40] 0.2568 - 0.2754 0.2175098524275899 0.2986356275755792
Sintetico: 0.3342 (0.1754) [100] 0.2998 - 0.3686 -0.0126531269997907 0.4538890507069509

Algorithm: CDSDG Clustering with n_clusters